In [26]:
# Step 1: Import necessary libraries
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

True

In [27]:
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [28]:
# Step 2: Load the PDF
pdf_path = "medical_book.pdf"  # Replace with your PDF file path
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} documents from the PDF.")

incorrect startxref pointer(1)
parsing for Object Streams


Loaded 637 documents from the PDF.


In [30]:
# Step 3: Create a FAISS vector store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embedding_model)
print("Vector store created.")

Vector store created.


In [31]:
# Step 4: Initialize LLM and create chains
llm = OpenAI(temperature=0.4, max_tokens=500)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})


# Create the retrieval chain with the correct syntax
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate

# Create a prompt template for the retrieval
prompt = PromptTemplate.from_template("""Answer the following question based on the provided context:

Context: {context}
Question: {input}

Answer:""")

# Create the retrieval chain with the prompt
retrieval_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=create_stuff_documents_chain(llm=llm, prompt=prompt)
)

print("Chains initialized.")

Chains initialized.


In [32]:
# Step 5: Query the RAG model
question = "What are the symptoms of diabetes?"
retrieved_docs = retrieval_chain.invoke({"input": question})  # Pass question as a dict with "input" key

print("\nResponse:")
print(retrieved_docs['answer'])


Response:
 Some common symptoms of diabetes include frequent urination, excessive thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, slow healing of cuts and bruises, and numbness or tingling in the hands and feet.
